# Beginning of the Official Jupyter Notebook for the Sports Bets Analyzer

In [ ]:
# The first thing I'd like to do is setup the API Data Collection

import pandas as pd
import numpy as np
import itertools

''' 
I will be using httpx instead of requests now and this link is the docs: https://www.python-httpx.org/
'''

def fetch_teams(sport, league, season):
    base_url = f'https://sports.core.api.espn.com/v2/sports/{sport}/leagues/{league}/seasons/{season}/teams'
    response = requests.get(base_url)
    
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error fetching teams for {sport} - {league}: {response.status_code}")
        return []

def fetch_team_roster(team_url):
    response = requests.get(team_url)
    
    if response.status_code == 200:
        data = response.json()
        print("Team Details Response:", data)  # Print the response for inspection
        return data
    else:
        print(f"Error fetching team roster: {response.status_code}")
        return {}

def fetch_player_details(player_url):
    response = requests.get(player_url)
    
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error fetching player details: {response.status_code}")
        return None



ModuleNotFoundError: No module named 'requests'

In [32]:
# Next I would like to setup the prop analyzer program so expected output based on given data by the API

def analyze_player_props(player_stats):
    player_props = []
    
    for player in player_stats:
        props = {
            "player_name": player.get('displayName', 'Unknown'),
            "points": player.get('points', np.nan),
            "rebounds": player.get('rebounds', np.nan),
            "assists": player.get('assists', np.nan),
            "td_scored": player.get('tds', np.nan),  # Example for football
            "expected_performance": player.get('expectedPerformance', np.nan)  # Placeholder for evaluation
        }
        player_props.append(props)

    return player_props

def generate_prop_combinations(player_props, min_picks=5, max_picks=6):
    all_combinations = []
    
    for num_picks in range(min_picks, max_picks + 1):
        if len(player_props) < num_picks:
            continue  # Skip if not enough players for this pick count
        
        combinations = itertools.combinations(player_props, num_picks)
        for combo in combinations:
            all_combinations.append({player['player_name']: player for player in combo})
    
    return all_combinations



In [34]:
# Second to last I want to create a program for choosing the best prop combinations
# from api import fetch_teams, fetch_team_roster
# from betting import analyze_player_props, generate_prop_combinations

# from api import fetch_teams, fetch_team_roster
# from betting import analyze_player_props, generate_prop_combinations
# import requests

def fetch_players(athletes_url):
    response = requests.get(athletes_url)
    
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error fetching athletes: {response.status_code}")
        return {}

def fetch_athlete_data(athlete_id):
    base_url = f"http://sports.core.api.espn.com/v2/sports/football/leagues/nfl/seasons/2024/athletes/{athlete_id}?lang=en&region=us"
    response = requests.get(base_url)
    
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Failed to fetch athlete data for ID {athlete_id}: {response.status_code}")
        return None

def main():
    sport = input("Enter the sport (e.g., football): ").strip().lower()
    league = input("Enter the league (e.g., nfl): ").strip().lower()
    season = input("Enter the season year (e.g., 2024): ").strip()
    
    teams_data = fetch_teams(sport, league, season)
    
    if teams_data:
        print("Teams in the selected league and season:")
        
        if 'items' in teams_data:
            for team_ref in teams_data['items']:
                team_url = team_ref['$ref']
                team_details = fetch_team_roster(team_url)

                if team_details:
                    team_name = team_details['displayName']
                    print(f"- {team_name}")
                else:
                    print("Team details could not be retrieved.")
        else:
            print("No items found in teams data.")

        selected_team_names = input("Enter the names of two teams separated by a comma (e.g., 'Denver Broncos, New Orleans Saints'): ")
        selected_teams = [name.strip().lower() for name in selected_team_names.split(",")]
        
        all_player_stats = []
        
        for team_ref in teams_data['items']:
            team_url = team_ref['$ref']
            team_details = fetch_team_roster(team_url)

            if team_details:
                team_name = team_details['displayName'].lower()
                if team_name in selected_teams:
                    print(f"Fetching roster for {team_details['displayName']}...")
                    athletes_url = team_details['athletes']['$ref']
                    players_data = fetch_players(athletes_url)
                    
                    # Fetch detailed stats for each player
                    if 'athletes' in players_data:
                        for player in players_data['athletes']:
                            player_ref = player['$ref']  # Get the player's reference link
                            player_id = player['uid'].split("~")[-1]  # Extract the athlete ID
                            player_details = fetch_athlete_data(player_id)  # Fetch detailed player stats
                            if player_details:
                                all_player_stats.append(player_details)  # Append detailed player data
                    else:
                        print(f"No athletes found for {team_details['displayName']}. Data received: {players_data}")
                else:
                    print(f"{team_details['displayName']} is not one of the selected teams.")

        if all_player_stats:
            player_props = analyze_player_props(all_player_stats)
            prop_combinations = generate_prop_combinations(player_props, min_picks=5, max_picks=6)

            # Display Results
            print("\nGenerated Props Combinations:")
            for bet in prop_combinations:
                print(bet)
        else:
            print("No player stats available for the selected teams.")
    else:
        print("No teams data available.")

if __name__ == "__main__":
    main()


Teams in the selected league and season:
Team Details Response: {'$ref': 'http://sports.core.api.espn.com/v2/sports/football/leagues/nfl/seasons/2024/teams/22?lang=en&region=us', 'id': '22', 'guid': '8fc7b962-95e1-3cb8-6a7d-b499de9ad546', 'uid': 's:20~l:28~t:22', 'alternateIds': {'sdr': '8801'}, 'slug': 'arizona-cardinals', 'location': 'Arizona', 'name': 'Cardinals', 'nickname': 'Cardinals', 'abbreviation': 'ARI', 'displayName': 'Arizona Cardinals', 'shortDisplayName': 'Cardinals', 'color': 'a40227', 'alternateColor': 'ffffff', 'isActive': True, 'isAllStar': False, 'logos': [{'href': 'https://a.espncdn.com/i/teamlogos/nfl/500/ari.png', 'width': 500, 'height': 500, 'alt': '', 'rel': ['full', 'default'], 'lastUpdated': '2024-06-25T18:43Z'}, {'href': 'https://a.espncdn.com/i/teamlogos/nfl/500-dark/ari.png', 'width': 500, 'height': 500, 'alt': '', 'rel': ['full', 'dark'], 'lastUpdated': '2024-06-25T18:43Z'}, {'href': 'https://a.espncdn.com/i/teamlogos/nfl/500/scoreboard/ari.png', 'width': 

In [30]:
# Lastly, we should have our main program 